# NASA Space Apps 2020
## Automated Detection of Hazards  
  
____________________  

Here we do predictions and save them to the DB

In [ ]:
import time
import datetime
import keras
import mysql.connector
import pandas as pd
import math

*Our columns: landslide_type	landslide_size	trigger	injuries	fatalities*

### Make predictions for next days

In [ ]:
model = keras.models.load_model("./model.hdf5")

## MYSQL

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="stati"
)
print(mydb)

## Points to predict

In [ ]:
#POINTS FOR DEMO
points = [
    [47.027963, 28.837133,"Moldova"],
    [47.756670, 27.908760,"Moldova"],
    [47.375748, 28.822655,"Moldova"],
    
    [50.400586, 30.451840,"Ukraine"],
    [49.811988, 23.997584,"Ukraine"],
    [49.391303, 26.991186,"Ukraine"],
    
    [44.337761, 26.009808,"Romania"],
    [46.585592, 23.492343,"Romania"],
    [44.250436, 23.753458,"Romania"]
]

## Prepare data info

In [ ]:
df=pd.read_csv("landslides.csv")
df["landslide_type"]=df["landslide_type"].str.lower()
df["landslide_size"]=df["landslide_size"].str.lower()
df["trigger"]=df["trigger"].str.lower()
df = df[df.landslide_type != "unknown"]
df = df[df.trigger != "unknown"]
df=df.fillna(0)
df = df[df.trigger != 0]

landslide_types = list(set(df['landslide_type']))
landslide_sizes = list(set(df['landslide_size']))
triggers = list(set(df['trigger']))

In [ ]:
#1st day
for p in points:
    data = model.predict(x=np.array([[1.001,p[0],p[1]]]))[0]
    mycursor = mydb.cursor()

    print(data)
    
    sql = "INSERT INTO redalert (la, lo, hazard, types, size, trig, injuries, fatalities, prob_trig, country) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (p[0],p[1],"landslides",landslide_types[math.floor(data[0])],landslide_sizes[math.floor(data[1])],triggers[math.floor(data[2])],str(data[3]),str(data[4]),data[2]-math.floor(data[2]),p[2])
    mycursor.execute(sql, val)

    mydb.commit()